In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, array_join

# -----------------------------
# 1. Join with diagnosis_ref
# -----------------------------
claims_with_desc = claims.join(
    diagnosis_ref,
    claims["DiagnosisCode"] == diagnosis_ref["Code"],
    how="left"
)

# -----------------------------
# 2. Join with members and providers for enrichment
# -----------------------------
claims_joined = claims_with_desc \
    .join(members.select("MemberID", "FirstName"), on="MemberID", how="left") \
    .join(providers.select("ProviderID", "Name"), on="ProviderID", how="left")

# -----------------------------
# 3. Aggregate back to one row per claim
# -----------------------------
claims_final = claims_joined.groupBy(
    "ClaimID", "MemberID", "ProviderID", "Amount",
    "ProcedureCode", "ClaimDate", "Status", "Remarks"
).agg(
    F.first("FirstName").alias("MemberName"),
    F.first("Name").alias("ProviderName"),
    F.collect_list("Description").alias("DiagnosisDescriptions")
)

# -----------------------------
# 4. Convert list to comma-separated string
# -----------------------------
claims_final = claims_final.withColumn(
    "DiagnosisDescriptions",
    array_join(col("DiagnosisDescriptions"), ",")
)

# -----------------------------
# 5. Fill nulls
# -----------------------------
claims_final = claims_final.fillna({
    "MemberName": "UNKNOWN",
    "ProviderName": "UNKNOWN",
    "DiagnosisDescriptions": "UNKNOWN"
})

# -----------------------------
# 6. Save Silver Table
# -----------------------------
claims_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("default.silver_claims_enriched")
